### Importação dos módulos a serem usados

##### Instalação de modulos

In [ ]:
# !pip install python-dotenv
# !pip install boto3
# !pip install pandas
# !pip install matplotlib
# !pip install opencv-python


In [ ]:
from dotenv import load_dotenv
import os
import boto3
import pandas as pd
import matplotlib.pyplot as plt
import time
import cv2
import re
import numpy as np

: 

#### Carregando variaveis de ambiente

In [19]:
load_dotenv()
ACCESS_ID = os.getenv("ACCESS_ID")
ACCESS_KEY = os.getenv("ACCESS_KEY")
REGIAO = "us-east-1"
S3_NAME = 'fiap-final-s3'

### pegar selfie via camera

In [20]:
def capture_selfie_with_frame(output_folder='selfie', frame_color=(0, 255, 0), frame_thickness=2):
    # Verifica se a pasta de saída existe, caso contrário, cria
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Inicializa a captura de vídeo
    cap = cv2.VideoCapture(0)
    
    while True:
        # Captura frame por frame
        ret, frame = cap.read()
        
        # Verifica se a captura foi bem-sucedida
        if not ret:
            print("Falha ao capturar imagem")
            break
        
        # Obtém as dimensões do frame
        height, width, _ = frame.shape
        
        # Define as coordenadas do centro do frame
        center_coordinates = (int(width / 2), int(height / 2))
        
        # Define os eixos do oval
        axes_length = (int(width * 0.17), int(height * 0.43))
        
        # Cria uma máscara para a elipse
        mask = np.zeros_like(frame)
        cv2.ellipse(mask, center_coordinates, axes_length, 0, 0, 360, (255, 255, 255), -1)
        
        # Aplica o desfoque na imagem inteira
        blurred_frame = cv2.GaussianBlur(frame, (21, 21), 5)
        
        # Combina a imagem desfocada com a área nítida dentro da elipse
        frame = cv2.bitwise_and(frame, mask)
        blurred_frame = cv2.bitwise_and(blurred_frame, cv2.bitwise_not(mask))
        frame = cv2.add(frame, blurred_frame)
        
        # Adiciona a mensagem para centralizar o rosto
        cv2.putText(frame, 'Centralize o rosto e aperte "q" para capturar', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Mostra o frame com a moldura
        cv2.imshow('Capture Selfie', frame)
        
        # Aguarda a tecla 'q' para capturar a imagem
        if cv2.waitKey(1) & 0xFF == ord('q'):
            # Captura apenas a área dentro da elipse
            x, y, w, h = center_coordinates[0] - axes_length[0], center_coordinates[1] - axes_length[1], axes_length[0] * 2, axes_length[1] * 2
            cropped_frame = frame[y:y+h, x:x+w]
            
            # Salva a imagem capturada
            img_name = os.path.join(output_folder, 'selfie.png')
            cv2.imwrite(img_name, cropped_frame)
            print(f"Imagem salva em {img_name}")
            break
    
    # Libera a captura e fecha todas as janelas
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# Chama a função para capturar a selfie
capture_selfie_with_frame()


Imagem salva em selfie/selfie.png


##### Preparação de salvamentos dos dados para o BucketS3 da AWS

In [21]:
# Função para subir arquivos para o S3 da aws
def upload_to_s3(file_name, bucket=S3_NAME , object_destination_name=None):
    # Se quiser alterar o nome no destino do arquivo utilizar o object_destination_name
    if object_destination_name is None:
        object_destination_name = file_name

    # Cria uma sessão do boto3
    session = boto3.Session(
        aws_access_key_id=ACCESS_ID,
        aws_secret_access_key=ACCESS_KEY,
        region_name=REGIAO
    )

    # Cria um cliente S3
    s3_client = session.client('s3')
    for retry in range(1,3):
        try:
            # Faz o upload do arquivo
            s3_client.upload_file(file_name, bucket, object_destination_name)
            print(f"Arquivo {file_name} carregado com sucesso para {bucket}/{object_destination_name}")
            break
        except Exception as e:
            print(f'Tentativa {retry} de 3')
            print(f"Erro ao carregar o arquivo: {e}")
            time.sleep(2**retry)


In [ ]:
folder_list = [ 'cnh', 'comprovante', 'selfie']

# Listar todos os arquivos na pasta IMG

for folder in list(folder_list):
    files = os.listdir(folder)
    # Enviar cada arquivo para o bucket do S3
    for file in files:
        file_path = os.path.join(folder, file)
        upload_to_s3(file_path, object_destination_name=file)

Arquivo cnh/cnh.png carregado com sucesso para fiap-final-s3/cnh.png
Arquivo comprovante/VIVO_FATURA_SEGUNDA_VIA.pdf carregado com sucesso para fiap-final-s3/VIVO_FATURA_SEGUNDA_VIA.pdf
Arquivo selfie/selfie.png carregado com sucesso para fiap-final-s3/selfie.png


#### Função extração foto, Nome, CPF

In [23]:
# Sessão AWS
session = boto3.Session(aws_access_key_id=ACCESS_ID, aws_secret_access_key= ACCESS_KEY)

# Extract text from image
client = session.client('textract', region_name=REGIAO)

cnh_s3 = {"S3Object": {"Bucket" : S3_NAME, "Name": "cnh.png"}}
comprovante_s3 = {"S3Object": {"Bucket" : S3_NAME, "Name": "comprovante.png"}}
selfie_s3 = {"S3Object": {"Bucket" : S3_NAME, "Name": "selfie.png"}}

In [24]:
response = client.analyze_document(Document=cnh_s3, FeatureTypes=['FORMS'])

In [25]:
print(response)

{'DocumentMetadata': {'Pages': 1}, 'Blocks': [{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 0.9919213056564331, 'Height': 0.9672924876213074, 'Left': 0.0010396104771643877, 'Top': 0.0}, 'Polygon': [{'X': 0.0010396104771643877, 'Y': 0.0}, {'X': 0.9811079502105713, 'Y': 0.03113769181072712}, {'X': 0.9929609298706055, 'Y': 0.9672924876213074}, {'X': 0.010944144800305367, 'Y': 0.9339594841003418}]}, 'Id': 'd41f4bcf-8ad2-4312-a6d5-ffb9d4afde7d', 'Relationships': [{'Type': 'CHILD', 'Ids': ['6da8015d-8d7d-4d05-b58f-c4c64e25291b', '6d49bf73-b92e-4172-860c-a4911f22177a', 'b54f1f04-8d59-46ea-b873-eda0d592b360', '1bb69ffd-4847-469e-9a77-e51578379e45', 'cc7aa6e3-d2cc-48da-b64e-736033808a32', 'a9371b9f-33a2-4802-91c4-6414b71e5dc2', '19adcfea-f055-4e38-ac13-e4074cf8149d', '34d53aac-3300-4836-8532-57556e6baed2', '04210765-bd92-463b-92aa-c2bab735a6cd', '4c2bd9c0-ae16-4313-b29c-c7dbe9799129', 'a3f71c13-06db-45ca-9741-c2356248adb2', 'e2c300d7-b473-4a9c-9a45-252f232e2aee', 'a8613e34-4451-4b

In [35]:
def find_nearest_text(blocks, target_text, search_below=True):
    """
    Procura um texto específico e encontra o bloco de texto mais próximo abaixo dele.
    """
    positions = []
    texts = {}

    # Mapear posições dos textos
    for block in blocks:
        if block['BlockType'] == 'LINE':
            text = block.get('Text', '')
            bbox = block['Geometry']['BoundingBox']
            y_center = bbox['Top'] + (bbox['Height'] / 2)  # Posição central Y

            texts[y_center] = text
            positions.append(y_center)

    positions.sort()  # Ordena as posições verticais
    if target_text == "Nome":
        found = False
        for i, y in enumerate(positions):
            if target_text.lower() in texts[y].lower():
                found = True
                continue
            if found:  # Pega o próximo elemento abaixo
                return texts[y] if search_below else texts[positions[i-1]]

        return "Não encontrado"
    if target_text == 'CPF':
        for key, value in texts.items():
            cpf_pattern = r"\b\d{3}\.\d{3}\.\d{3}-\d{2}\b"
            match = re.search(cpf_pattern, value)
            if match:
                return value
        return "CPF não encontrado"
    


# Extrair blocos de texto
blocks = response['Blocks']

# Buscar nome e CPF
nome = find_nearest_text(blocks, "Nome")
cpf = find_nearest_text(blocks, "CPF")

print(f"Nome: {nome}")
print(f"CPF: {cpf}")

# Extrair blocos de texto


Nome: VINICIUS MATTEONI DAS DORES
CPF: CPF não encontrado


In [28]:
print(blocks)

[{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 0.9919213056564331, 'Height': 0.9672924876213074, 'Left': 0.0010396104771643877, 'Top': 0.0}, 'Polygon': [{'X': 0.0010396104771643877, 'Y': 0.0}, {'X': 0.9811079502105713, 'Y': 0.03113769181072712}, {'X': 0.9929609298706055, 'Y': 0.9672924876213074}, {'X': 0.010944144800305367, 'Y': 0.9339594841003418}]}, 'Id': 'd41f4bcf-8ad2-4312-a6d5-ffb9d4afde7d', 'Relationships': [{'Type': 'CHILD', 'Ids': ['6da8015d-8d7d-4d05-b58f-c4c64e25291b', '6d49bf73-b92e-4172-860c-a4911f22177a', 'b54f1f04-8d59-46ea-b873-eda0d592b360', '1bb69ffd-4847-469e-9a77-e51578379e45', 'cc7aa6e3-d2cc-48da-b64e-736033808a32', 'a9371b9f-33a2-4802-91c4-6414b71e5dc2', '19adcfea-f055-4e38-ac13-e4074cf8149d', '34d53aac-3300-4836-8532-57556e6baed2', '04210765-bd92-463b-92aa-c2bab735a6cd', '4c2bd9c0-ae16-4313-b29c-c7dbe9799129', 'a3f71c13-06db-45ca-9741-c2356248adb2', 'e2c300d7-b473-4a9c-9a45-252f232e2aee', 'a8613e34-4451-4bf7-816f-f3d0ea624324', '7490dc5a-a71a-4eff-8a